Here we find the Python scripts used in paper 'User-level Handover Decision Making Based on Machine Learning' written by João PSH Lima et al. The intention is to apply classification algorithms to indicate the best handover target in a LTE cells urban environment. Additionally, we also apply regression algorithms to estimate file download time and percentage, so that handover decision making is based upon richer information conditions and network designers may have enhanced performance. First, we consider Scenario 1, without shadowing effects.

The classification methods used in this comparison are:

- SOFL
- ALMMo
- T2
- SVM
- MLP

The classification of the best handover target is based upon 3 rules, which are:

- The best handover target is the one that completes the download;
- If both eNBs are able to complete the download (download time < simulation time), the best is the one that does it faster;
- If both eNBs are not able to complete the download (download time = simulation time), the best target is the one that downloaded more bytes from file.

These rules are expressed in function below:

In [4]:
# Let us make the imports for the entire code

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix
import time
# Enable to start counting processing time
# start = time.time()

def best_output(row):
    # if downloadTime is different than 100 for T2 and T3, both have completed the download
    if ((row.downloadTimeT2!=100)&(row.downloadTimeT3!=100)):
        # the best output has smaller downloadTime
        if (row.downloadTimeT2<=row.downloadTimeT3):
            return 2
        else:
            return 1

    # if downloadTime is different than 100 for only one target, only one completes the download
    elif ((row.downloadTimeT2!=100)|(row.downloadTimeT3!=100)):
        # the best output has downloadTime other than 100 (completed download before simulation time ends)
        if (row.downloadTimeT2!=100):
            return 2
        else:
            return 1

    # if downloadTime = 100 for both T2 and T3, both targets does not complete download
    elif ((row.downloadTimeT2==100)&(row.downloadTimeT3==100)):
        # the best output has greater rxBytes
        if (row.rxBytesT2>=row.rxBytesT3):
            return 2
        else:
            return 1

In [6]:
# Here we read and organize CSV data

t2 = pd.read_csv('t2_OkumuraHata_Modificado', delimiter='\t')
t3 = pd.read_csv('t3_OkumuraHata_Modificado', delimiter='\t')

# Guarantee that we utilize only seeds present in both datasets
t2 = t2[t2.nRun.isin(t3.nRun)]
t3 = t3[t3.nRun.isin(t2.nRun)]
t2 = t2.reset_index(drop=True)
t3 = t3.reset_index(drop=True)

# Combining datasets
data = t2
data = data.drop(['targetCellId', 'downloadTime', 'rxBytes'], axis=1)
data['downloadTimeT2'] = t2.downloadTime
data['downloadTimeT3'] = t3.downloadTime
data['rxBytesT2'] = t2.rxBytes
data['rxBytesT3'] = t3.rxBytes

In [7]:
# Data Pre-processing

# Applies our function defined above to obtain the best output
data['best_output'] = data.apply(best_output, axis=1)

# Sets data as inputs and labels
previsores = data[['rsrp1','rsrq1','rsrp2','rsrq2','rsrp3','rsrq3','previousrsrp1','previousrsrq1','previousrsrp2','previousrsrq2','previousrsrp3','previousrsrq3']]
previsores = previsores.values
label = data['best_output']
label = label.values

# Scaling data
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

In [15]:
# Now we load our classifier, execute k-Fold, train and test our algorithm

# ALMMo
from ALMMo0_System import ALMMo0classifier_testing
from ALMMo0_System import ALMMo0classifier_learning

# Applies StratifiedKFold with k = 5 and repeats process 33 times for statistical robustness
resultados33 = []

for i in range(33):
    kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = i)
    resultados1 = []
    matriz1 = [] # confusion matrix
    for n_train, n_test in kfold.split(previsores, np.zeros(shape=(previsores.shape[0], 1))):
        # Train the ALMMo-0 classifier
        SystemParameters=ALMMo0classifier_learning(previsores[n_train], label[n_train])
         # Conduct classification with the pretrained classifier
        TestLabel=ALMMo0classifier_testing(previsores[n_test],SystemParameters)
        # Accuracy check
        precisao = accuracy_score(label[n_test], TestLabel)
        matriz1.append(confusion_matrix(label[n_test], TestLabel))
        resultados1.append(precisao)
    # Appending all the steps
    resultados1 = np.asarray(resultados1)
    media = resultados1.mean()
    resultados33.append(media)
    matriz_final = np.mean(matriz1, axis = 0)
# Final results
resultados33 = np.asarray(resultados33)

# Enable to obtain processing time
# end = time.time()
# tempo = end - start

# Enable to display classification mean and standard deviation
resultados33.mean()
# resultados33.std()

0.9960606060606062

In [14]:
# SVM classifier
from sklearn.svm import SVC

# Applies StratifiedKFold with k = 5 and repeats process 33 times for statistical robustness
resultados33 = []

# 33-time loop
for i in range(33):
    kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = i)
    resultados1 = []
    matriz1 = []
    # k-Fold loop
    for n_train, n_test in kfold.split(previsores, np.zeros(shape=(previsores.shape[0], 1))):
        svm = SVC(kernel = 'linear', C = 10.0)
        # Training model
        svm.fit(previsores[n_train], label[n_train])
        # Predicting
        previsoes = svm.predict(previsores[n_test])
        # Accuracy check
        precisao = accuracy_score(label[n_test], previsoes)
        matriz1.append(confusion_matrix(label[n_test], previsoes))
        resultados1.append(precisao)
    # Appending all the steps
    resultados1 = np.asarray(resultados1)
    media = resultados1.mean()
    resultados33.append(media)
    matriz_final = np.mean(matriz1, axis = 0)
# Final results
resultados33 = np.asarray(resultados33)

# Enable to obtain processing time
# end = time.time()
# tempo = end - start

# Enable to display classification mean and standard deviation
resultados33.mean()
# resultados33.std()

0.9974242424242424

In [17]:
# MLP classifier
from sklearn.neural_network import MLPClassifier

# Applies StratifiedKFold with k = 5 and repeats process 33 times for statistical robustness
resultados33 = []

# 33-time loop
for i in range(33):
    kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = i)
    resultados1 = []
    matriz1 = []
    # k-Fold loop
    for n_train, n_test in kfold.split(previsores, np.zeros(shape=(previsores.shape[0], 1))):
        nn3 = MLPClassifier(solver='lbfgs', hidden_layer_sizes = 6)
        # Training model
        nn3.fit(previsores[n_train], label[n_train])
        # Predicting
        previsoes = nn3.predict(previsores[n_test])
        # Accuracy check
        precisao = accuracy_score(label[n_test], previsoes)
        matriz1.append(confusion_matrix(label[n_test], previsoes))
        resultados1.append(precisao)
    # Appending all the steps
    resultados1 = np.asarray(resultados1)
    media = resultados1.mean()
    resultados33.append(media)
    matriz_final = np.mean(matriz1, axis = 0)
# Final results
resultados33 = np.asarray(resultados33)

# Enable to obtain processing time
# end = time.time()
# tempo = end - start

# Enable to display classification mean and standard deviation
resultados33.mean()
# resultados33.std()

0.9973636363636365